In [1]:
import pickle
import os

In [2]:
def load_circuit_stats(name):
    with open('./feyn_bench/stats/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def compare_circuits(c1_data, c2_data):
    for criterion in ['T-gates','2-qubit','1-qubit']:
        if c1_data[criterion] < c2_data[criterion]:
            return 'g'
        if c1_data[criterion] > c2_data[criterion]:
            return 'l'
    return 'e' #completely equal
    
def get_best_heuristic_circuit(circuit_data):
    best_entry = {'idx': 0, 'data': circuit_data[0]}
    for idx,entry in enumerate(circuit_data[3:]): #skip original, basic_opt and pyzx
        if compare_circuits(entry, best_entry['data']) == 'g':
            best_entry = {'idx': idx, 'data': entry}
    return best_entry['data']
    
def get_circuit_data(name, todd=False):
    if todd:
        data = load_circuit_stats(name+'_t')
    else:
        data = load_circuit_stats(name)
    return (data[0],data[2],get_best_heuristic_circuit(data))

def merge_circuit_data(name, todd=False, cols=['1-qubit','2-qubit','T-gates']):
    merged_data = []
    orig, pyzx, heur = get_circuit_data(name, todd)
    merged_data = [orig] #name orig, 
    for routine in [pyzx, heur]:
        for col in cols:
            merged_data.append(routine[col])
    merged_data.append(heur['strategy'].replace('_','\symbol{95}'))
    return merged_data

def format_latex_row(row):
    latex_str = ""
    for entry in row[:-1]:
        latex_str += str(entry) + ' & '
    latex_str += str(row[-1]) + ' \\'
    return latex_str

def print_results(output_format):
    circuit_table = []
    for file in os.listdir('./feyn_bench/before/'):
        if os.path.isfile('./feyn_bench/stats/' + file + '.pkl'):
            circuit_table.append(merge_circuit_data(file, todd=False))
    if output_format == 'latex':
        latex_str = '' #'1-orig & 2-orig & T-orig & 1-pyzx & 2-pyzx & T-pyzx & 1-heur & 2-heur & T-heur \\ \n'
        for circuit in circuit_table:
            latex_str += format_latex_row(circuit) + ' sep'
        return latex_str
    return circuit_table
    

In [ ]:
print_results('latex')